In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d84e6339967e401bc13b0d20d8b04354422c33e6d0d723b1004997dc03ee4222
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
from pyspark import SparkContext
sc = spark.sparkContext

In [4]:
df = spark.read.format("csv") \
.option("inferSchema", True) \
.option("sep",",") \
.load("leaf.csv")

df.toPandas().head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15
0,1,1,0.72694,1.4742,0.32396,0.98535,1.00000,0.83592,0.004657,0.003947,0.047790,0.127950,0.016108,0.005232,0.000275,1.17560
1,1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.005242,0.005002,0.024160,0.090476,0.008119,0.002708,0.000075,0.69659
2,1,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.007457,0.010121,0.011897,0.057445,0.003289,0.000921,0.000038,0.44348
3,1,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.006877,0.008607,0.015950,0.065491,0.004271,0.001154,0.000066,0.58785
4,1,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.007428,0.010042,0.007938,0.045339,0.002051,0.000560,0.000024,0.34214


In [5]:
df.describe().toPandas()

,summary,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15
0,count,340,340,340,340,340,340,340,340,340,340,340,340,340,340,340,340
1,mean,18.544117647058822,6.2823529411764705,0.7198541764705882,2.44021,0.5137604117647057,0.9041577352941174,0.9437929705882349,0.5312335529411764,0.037344603235294115,0.5238449391176467,0.051346355588235325,0.1245348205882353,0.0176699794117647,0.0059277176176470575,3.872229714705881E-4,1.162629882352942
2,stddev,11.152514463795796,3.4627789570161087,0.20831105515953707,2.599042669706628,0.1955829569366353,0.11463914421737179,0.11504654762850589,0.21753156635197998,0.03857491901164008,1.0396394142951784,0.03596491370502356,0.05185961725619335,0.013755046250182099,0.0052941214059131665,4.3144007160996904E-4,0.5848537417736618
3,min,1,1,0.11708,1.0066,0.10761,0.48549,0.39649,0.078376,0.0028365,0.0014643,0.0050219,0.033415,0.0011153,2.2942E-4,6.9241E-6,0.1694
4,max,36,16,0.99871,19.038,0.94834,0.99388,1.0,0.85816,0.19898,7.2062,0.19067,0.28081,0.073089,0.029786,0.0029358,2.7085


**No missing values.**


In [6]:
import pyspark.sql.functions as f
df.groupBy("_c0").agg(f.count("*").alias("count")).show()

+---+-----+
|_c0|count|
+---+-----+
| 31|   11|
| 34|   11|
| 28|   12|
| 26|   12|
| 27|   11|
| 12|   12|
| 22|   12|
|  1|   12|
| 13|   13|
|  6|    8|
|  3|   10|
|  5|   12|
| 15|   10|
|  9|   14|
| 35|   11|
|  4|    8|
|  8|   11|
| 23|   11|
|  7|   10|
| 10|   13|
+---+-----+
only showing top 20 rows



In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [8]:
df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15']

In [9]:
df = df.withColumnRenamed('_c0','label')
df = df.drop("_c1")
df.toPandas().head()

,label,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15
0,1,0.72694,1.4742,0.32396,0.98535,1.00000,0.83592,0.004657,0.003947,0.047790,0.127950,0.016108,0.005232,0.000275,1.17560
1,1,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.005242,0.005002,0.024160,0.090476,0.008119,0.002708,0.000075,0.69659
2,1,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.007457,0.010121,0.011897,0.057445,0.003289,0.000921,0.000038,0.44348
3,1,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.006877,0.008607,0.015950,0.065491,0.004271,0.001154,0.000066,0.58785
4,1,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.007428,0.010042,0.007938,0.045339,0.002051,0.000560,0.000024,0.34214


In [10]:
assembler = VectorAssembler() \
.setInputCols([
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15']) \
.setOutputCol("vectorized_features")

In [11]:
X_train, X_test = df.randomSplit([0.7, 0.3], seed=42)

In [12]:
def params_for_all(k):
  if k == 0:
    param_grid = ParamGridBuilder() \
  .addGrid(clf_model.aggregationDepth, [3,5,7,10]) \
  .addGrid(clf_model.elasticNetParam, [0.0, 0.2, 0.7, 0.9]) \
  .addGrid(clf_model.maxIter, [100, 200, 300]) \
  .addGrid(clf_model.regParam, [0.00, 0.01, 0.05]) \
  .addGrid(clf_model.tol, [1.0E-6, 1.0E-4]) \
  .build()

  elif k == 1:
    param_grid = ParamGridBuilder() \
  .addGrid(clf_model.impurity, ["entropy", "gini"]) \
  .addGrid(clf_model.maxBins, [32,64,100]) \
  .addGrid(clf_model.maxDepth, [2,5,7,10,15,20]) \
  .build()

  elif k == 2:
    param_grid = ParamGridBuilder() \
  .addGrid(clf_model.impurity, ["entropy", "gini"]) \
  .addGrid(clf_model.maxBins, [32,64,100,200,300]) \
  .addGrid(clf_model.maxDepth, [2,5,7,10,15,20,25]) \
  .addGrid(clf_model.numTrees, [20,30,40,50,60,80,100]) \
  .addGrid(clf_model.subsamplingRate, [0.5, 0.75, 0.9, 1.0]) \
  .build()

  return param_grid

In [13]:
def adding_bests(k):
  if k == 0:
    param_list.append(["aggregationDepth=", besty.getAggregationDepth(),"elasticNetParam=", besty.getElasticNetParam(),"MaxIter=", besty.getMaxIter(),
                    "regParam=", besty.getRegParam(),"tol=", besty.getTol()])

  elif k == 1:
    param_list.append(["impurity=", besty.getImpurity(),"maxBins=", besty.getMaxBins(),"maxDepth=", besty.getMaxDepth()])

  elif k == 2:
    param_list.append(["impurity=", besty.getImpurity(),"maxBins=", besty.getMaxBins(),"maxDepth=", besty.getMaxDepth(), "numTrees=", besty.getNumTrees, "subsamplingRate=", besty.getSubsamplingRate()])

# BaseLine Models

In [20]:
model_names = ["LogisticRegression","DecisionTreeClassifier","RandomForestClassifier"]
accuracy_list = []
param_list = []
model_type = []

clfs = [LogisticRegression(),DecisionTreeClassifier(),RandomForestClassifier()]

model = 0
for i in clfs:
  clf_model = i.setFeaturesCol("vectorized_features").setLabelCol("label").setPredictionCol("prediction")

  pipeline_x = Pipeline().setStages([assembler,clf_model])

  param_grid = ParamGridBuilder().build()

  acc_eva = MulticlassClassificationEvaluator(metricName="accuracy")
  cv = CrossValidator().setEstimator(pipeline_x).setEvaluator(acc_eva).setEstimatorParamMaps(param_grid).setNumFolds(3).setParallelism(8)
  cv_model = cv.fit(X_train)
  resultDF = cv_model.transform(X_test)
  acc_score = acc_eva.evaluate(resultDF)
  besty = cv_model.bestModel.stages[-1]
  param_list.append(["Baseline Models No Tuning"])
  model_type.append(model_names[model])
  accuracy_list.append(acc_score)
  model += 1

# Models With Hyperparameter Tuning

In [22]:
k = 0
for i in clfs:
  clf_model = i.setFeaturesCol("vectorized_features").setLabelCol("label").setPredictionCol("prediction")

  pipeline_x = Pipeline().setStages([assembler,clf_model])

  param_grid = params_for_all(k)

  acc_eva = MulticlassClassificationEvaluator(metricName="accuracy")
  cv = CrossValidator().setEstimator(pipeline_x).setEvaluator(acc_eva).setEstimatorParamMaps(param_grid).setNumFolds(3).setParallelism(8)
  cv_model = cv.fit(X_train)
  resultDF = cv_model.transform(X_test)
  acc_score = acc_eva.evaluate(resultDF)
  besty = cv_model.bestModel.stages[-1]
  adding_bests(k)
  model_type.append(model_names[k])  
  accuracy_list.append(acc_score)
  k += 1

# Results

In [23]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

columns = ['Model_Name', 'Accuracy', 'Best Params']

result_df = spark.createDataFrame(
  zip(model_type, accuracy_list, param_list), columns)

result_df.sort(f.col("Accuracy").desc()).toPandas()

,Model_Name,Accuracy,Best Params
0,LogisticRegression,0.735632,[Baseline Models No Tuning]
1,LogisticRegression,0.735632,"[aggregationDepth=, 3, elasticNetParam=, 0.0, MaxIter=, 100, regParam=, 0.0, tol=, 1.0E-6]"
2,RandomForestClassifier,0.678161,"[impurity=, entropy, maxBins=, 64, maxDepth=, 7, numTrees=, 50, subsamplingRate=, 1.0]"
3,DecisionTreeClassifier,0.620690,"[impurity=, entropy, maxBins=, 32, maxDepth=, 7]"
4,RandomForestClassifier,0.609195,[Baseline Models No Tuning]
5,DecisionTreeClassifier,0.344828,[Baseline Models No Tuning]
